### Collecting some ping statistics

Ping data collected every 5 seconds doing single pings to 8.8.8.8

**Example**:
>$ ping -c 1 8.8.8.8  
>PING 8.8.8.8 (8.8.8.8): 56 data bytes  
>64 bytes from 8.8.8.8: icmp_seq=0 ttl=119 **time=22.985 ms**  
>--- 8.8.8.8 ping statistics ---  
>1 packets transmitted, 1 packets received, **0.0% packet loss**  
>round-trip min/**avg**/max/**stddev** = 22.985/22.985/**22.985**/**0.000** ms  

Data beeing collected:
 - **Ping latency** (in miliseconds, 22.985 ms in the example)
 - __Percentage of packets dropped__  (0.0% in the example, can be only 0% or 100%, data collected for single pings) 
 - **Standart deviation (for latency)**  - always 0 for single pings


In [1]:
from influxdb import DataFrameClient
from influxdb import InfluxDBClient
import json

from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go
import plotly.plotly as py
from plotly import tools
init_notebook_mode(connected=True)

Read credentials:

In [2]:
with open('../credentials.json', 'r') as f_credentials:
    credentials_config = json.load(f_credentials)

Set up influxdb connection:

In [3]:
host=credentials_config['influxdb_host']
port=8086
dbname = 'net_speed_md'
client = InfluxDBClient(host, port, '', '', dbname)

In [4]:
#Trying to load entire table, convert to dataframe and estimate how much memory its taking
#from here https://stackoverflow.com/questions/18089667/how-to-estimate-how-much-memory-a-pandas-dataframe-will-need
#client = DataFrameClient(host, port, '', '', dbname)
#query_ping = 'SELECT * FROM PING;'
#result_ping = client.query(query_ping)
#ping_df = result_ping['PING']
#print(ping_df.memory_usage())
#print(sys.getsizeof(ping_df))
#rint( df.memory_usage(deep=True).sum())

Checking last 20 records in the ping measurment and see how data looks like:

In [5]:
client_df = DataFrameClient(host, port, '', '', dbname)
query_ping = 'SELECT PING,SK_PI FROM PING ORDER BY time DESC LIMIT 10;'
result_ping = client_df.query(query_ping)
ping_df = result_ping['PING']
ping_df

,PING,SK_PI
2019-01-01 23:59:57.196999936+00:00,0.000,12
2019-01-01 23:59:58.196999936+00:00,0.000,18
2019-01-01 23:59:58.196999936+00:00,0.000,9
2019-01-01 23:59:58.196999936+00:00,0.000,7
2019-01-01 23:59:58.196999936+00:00,0.000,15
2019-01-01 23:59:58.196999936+00:00,38.004,16
2019-01-01 23:59:59.200000+00:00,0.000,11
2019-01-01 23:59:59.200000+00:00,44.217,17
2019-01-01 23:59:59.200000+00:00,0.000,3
2019-01-01 23:59:59.200000+00:00,0.000,12


Let's take just one device, for example 3:

In [6]:
query_ping = "SELECT PING FROM PING WHERE SK_PI='3' ORDER BY time  DESC LIMIT 10 ;"
result_ping = client_df.query(query_ping)
ping_df = result_ping['PING']
ping_df

,PING
2019-01-01 23:59:41.180000+00:00,0.000
2019-01-01 23:59:43.183000064+00:00,14.573
2019-01-01 23:59:45.187000064+00:00,0.000
2019-01-01 23:59:47.187000064+00:00,14.690
2019-01-01 23:59:49.190000128+00:00,0.000
2019-01-01 23:59:51.190000128+00:00,0.000
2019-01-01 23:59:53.192999936+00:00,14.670
2019-01-01 23:59:55.196999936+00:00,0.000
2019-01-01 23:59:57.196999936+00:00,14.804
2019-01-01 23:59:59.200000+00:00,0.000


Lets compare with what we have in mysql database.

In [54]:
import pyodbc
import pandas as pd
password=credentials_config['mssql_password']
srv=credentials_config['mssql_host']
cnxn = pyodbc.connect(driver='/usr/local/lib/libtdsodbc.so', server=srv,port='1433', database='net_speed_md', uid='cybera_sql', pwd=password)
sql = "SELECT TOP 10 DATA_DATE, PING FROM FCT_PI where sk_pi='3' ORDER BY DATA_DATE DESC;"
data = pd.read_sql(sql,cnxn)
data

,DATA_DATE,PING
0,2019-01-02 23:59:59.600,14.668
1,2019-01-02 23:59:57.597,NaN
2,2019-01-02 23:59:55.597,14.791
3,2019-01-02 23:59:53.593,NaN
4,2019-01-02 23:59:51.590,NaN
5,2019-01-02 23:59:49.590,14.648
6,2019-01-02 23:59:47.587,NaN
7,2019-01-02 23:59:45.587,14.762
8,2019-01-02 23:59:43.583,NaN
9,2019-01-02 23:59:41.580,NaN


Pings are coming every 4-6 seconds. Missing values are NaN in original and 0 in the influxdb (we might consider re-importing and omitting values with NaN)

Checking statistics for number of packets dropped:

In [8]:
query_ping = "SELECT DISTINCT(PING_DROPRATE) from PING;"
result_ping = client_df.query(query_ping)
ping_df = result_ping['PING']
ping_df

,distinct
1970-01-01 00:00:00+00:00,0
1970-01-01 00:00:00+00:00,100


In Grafana we see that the device #10 has  some packets dropped, lets see what is the value of *PING* measurment when *PING_DROPRATE* is equal to 100:

In [9]:
query_ping = "SELECT PING, PING_DROPRATE FROM PING WHERE SK_PI='10' ORDER BY time DESC LIMIT 10;"
result_ping = client_df.query(query_ping)
ping_df = result_ping['PING']
ping_df

,PING,PING_DROPRATE
2018-12-31 04:39:45.187000064+00:00,46.312,0
2018-12-31 04:39:47.187000064+00:00,0.000,0
2018-12-31 04:39:49.190000128+00:00,46.767,0
2018-12-31 04:39:51.190000128+00:00,0.000,0
2018-12-31 04:39:53.192999936+00:00,0.000,0
2018-12-31 04:39:55.196999936+00:00,43.976,0
2018-12-31 04:39:57.196999936+00:00,0.000,0
2018-12-31 04:39:59.200000+00:00,46.056,0
2018-12-31 04:40:01.200000+00:00,0.000,0
2018-12-31 04:40:03.203000064+00:00,0.000,0


*PING* is equal to 0 in this case, its importatnt to note.

### Number of datapoints per device

Getting device numbers(tags SK_PI):

In [10]:
query_unique_devices = "SHOW TAG VALUES FROM PING WITH KEY=SK_PI;"
result_unique_devices = client.query(query_unique_devices)
points_unique_devices = result_unique_devices.get_points()
device_numbers=[]
for point in points_unique_devices:
    device_numbers.append(point['value'])
device_numbers=list(map(int, device_numbers))
device_numbers= sorted(device_numbers)
print(device_numbers)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]


Getting number of data points per device for the entire period of time.

In [11]:
query_ping_counts = 'SELECT COUNT(PING) FROM PING WHERE PING!=0 GROUP BY SK_PI;'
result_ping_counts = client.query(query_ping_counts)

In [12]:
query_pingdroprate_counts = 'SELECT COUNT(PING_DROPRATE) FROM PING WHERE PING_DROPRATE!=0 GROUP BY SK_PI;'
result_pingdroprate_counts = client.query(query_pingdroprate_counts)

In [13]:
ping_counts=[]
pingdroprate_counts=[]
for device in device_numbers:
    points_ping_counts=result_ping_counts.get_points(tags={'SK_PI':str(device)})
    points_pingdroprate_counts=result_pingdroprate_counts.get_points(tags={'SK_PI':str(device)})
    for point in points_ping_counts:
        count_ping=point['count']
    for point in points_pingdroprate_counts:
        count_pingdroprate=point['count']
    ping_counts.append(count_ping+count_pingdroprate)
    pingdroprate_counts.append(count_ping+count_pingdroprate)

Plotting device numbers and number of data points. 

In [14]:
data = [go.Bar(
            x=device_numbers,
            y=ping_counts
    )]

layout = go.Layout(
        barmode='stack',
        title="Number of data points per device"
    )

fig = go.Figure(data=data, layout=layout)
iplot(fig)

Some of the devices have small number of datapoints, may be they are just installed? Lets check how many dataponts came in last 4 weeks.

Getting number of datapoints per device in last 4 weeks and list of the devices that have data for this period of time(not all of the have).

In [15]:
#query_ping_counts_dec = "SELECT COUNT(PING) FROM PING WHERE time >= '2018-12-01 00:00:00' GROUP BY SK_PI ;"
query_ping_counts_dec = "SELECT COUNT(PING) FROM PING WHERE time >= now()-4w  AND PING!=0 GROUP BY SK_PI ;"
result_ping_counts_dec = client.query(query_ping_counts_dec)

In [16]:
query_pingdroprate_counts_dec = 'SELECT COUNT(PING_DROPRATE) FROM PING WHERE time >= now()-4w AND PING_DROPRATE!=0 GROUP BY SK_PI;'
result_pingdroprate_counts_dec = client.query(query_pingdroprate_counts_dec)

In [17]:
ping_counts_dec=[]
device_numbers_dec = []
pingdroprate_counts_dec = []
#ping_sent_counts_dec = []
for device in device_numbers:
    points_ping_counts_dec=result_ping_counts_dec.get_points(tags={'SK_PI':str(device)})
    points_pingdroprate_counts_dec=result_pingdroprate_counts_dec.get_points(tags={'SK_PI':str(device)})
    point_new=0
    point_ping=0
    point_pingdroprate=0
    #print(device)
    for point in points_ping_counts_dec:
        point_ping=point['count']
    for point in points_pingdroprate_counts_dec:
        point_pingdroprate=point['count']
    #print(point_ping)
    #print(point_pingdroprate)
    point_new=point_pingdroprate+point_ping
    #print(point_new)
    if (point_new!=0): 
        device_numbers_dec.append(device)
    ping_counts_dec.append(point_new)
    pingdroprate_counts_dec.append(point_pingdroprate)
    #ping_sent_counts_dec.append(point_ping)
print("Devices, that have data in last 4 weeks: ",device_numbers_dec)

Devices, that have data in last 4 weeks:  [3, 7, 9, 10, 11, 12, 15, 16, 17, 18]


Plotting combined barchart - entire number of datapoints vs number of datapoints in last 4 weeks.

In [18]:
trace1 = go.Bar(
            x=device_numbers,
            y=ping_counts_dec,
            name='Last 4 weeks',
    )
trace2 = go.Bar(
            x=device_numbers,
            y=[a - b for a, b in zip(ping_counts, ping_counts_dec)],
            name='Entire time',
    
    )
data = [trace1, trace2]
layout = go.Layout(
        barmode='stack',
        title="Comparing number of datapoints in last 4 weeks vs entire time"
    )

fig = go.Figure(data=data, layout=layout)
iplot(fig)

Devices 1,2,4,5,6,8 and possibly 13 and 14 need to be double checked. Looks like they have started reporting and then stopped. Let's check last reporting time for every device.

In [19]:
query_ping_last = "SELECT last(PING), time FROM PING WHERE PING!=0 OR PING_DROPRATE!=0 GROUP BY SK_PI;"
result_ping_last = client.query(query_ping_last)

In [20]:
for device in device_numbers:
    points_ping_last=result_ping_last.get_points(tags={'SK_PI':str(device)})
    for point in points_ping_last:
        print("Device: ", device," reported last time on ", point['time'])

Device:  1  reported last time on  2018-10-11T15:46:31.790000128Z
Device:  2  reported last time on  2018-11-06T19:16:32.792999936Z
Device:  3  reported last time on  2019-01-01T23:59:57.196999936Z
Device:  4  reported last time on  2018-12-05T22:53:02.383000064Z
Device:  5  reported last time on  2018-11-05T21:32:30.550000128Z
Device:  6  reported last time on  2018-10-31T14:55:00.7Z
Device:  7  reported last time on  2019-01-01T23:59:56.196999936Z
Device:  8  reported last time on  2018-12-03T19:58:50.129999872Z
Device:  9  reported last time on  2019-01-01T23:59:56.196999936Z
Device:  10  reported last time on  2018-12-31T04:39:59.2Z
Device:  11  reported last time on  2019-01-01T23:59:57.196999936Z
Device:  12  reported last time on  2019-01-01T23:59:55.196999936Z
Device:  13  reported last time on  2018-12-04T20:27:05.227000064Z
Device:  14  reported last time on  2018-12-06T02:12:27.347000064Z
Device:  15  reported last time on  2019-01-01T23:59:56.196999936Z
Device:  16  reporte

Something is happening with devices  1,2,4,5,6,8,13 and 14. They stopped reporting - does it need to be investigated?

## Ping latency 

What is normal ping latency? From [this link](https://www.pingman.com/kb/article/what-s-normal-for-latency-and-packet-loss-42.html):

>There are two normal factors that significantly influence the latency of a consumer device (like a cable modem, dsl modem or dial-up modem).

>The latency of the connecting device. For a cable modem, this can normally be between 5 and 40 ms. For a DSL modem this is normally 10 to 70ms. For a dial-up modem, this is normally anywhere from 100 to 220ms. For a cellular link, this can be from 200 to 600 ms. For a T1, this is normally 0 to 10 ms.
The distance the data is traveling. Data travels at (very roughly) 120,000 miles (or 192,000 kilometers) per second, or 120 miles (192 km) per ms (millisecond) over a network connection. With traceroute, we have to send the data there and back again, so roughly 1 ms of latency is added for every 60 miles (96km, although with the level of accuracy we're using here, we should say '100km') of distance between you and the target.
Connecting to a web site across 1500 miles (2400 km) of distance is going to add at least 25 ms to the latency. Normally, it's more like 75 after the data zig-zags around a bit and goes through numerous routers.

>This means that a DSL modem on the west coast of the United States, tracing to a server on the east coast of the United States should expect somewhere around 120 ms (depending on the route and a number of other factors, but this is a rough ballpark) - 25 ms for the DSL modem and 100 ms for the distance. Tracing across an ocean, or through a satellite link, or some other link where the distance is further will certainly impact the expected latency more.



Let's check actual ping latency numbers in last 4 weeks:

In [21]:
#query_ping_max_dec = "SELECT MAX(PING), MEAN(PING) FROM PING WHERE time >= '2018-12-01 00:00:00' GROUP BY SK_PI;"
query_ping_max_dec = "SELECT MAX(PING), MEAN(PING), MEDIAN(PING) FROM PING WHERE PING!=0 AND  time >= now()-4w  GROUP BY SK_PI;"
result_ping_max_dec = client.query(query_ping_max_dec)

We will exclude devices  1,2,4,5,6,8,13,14 as they are not reporting any more

In [22]:
#device_numbers_dec = [x for x in device_numbers if x not in [1,2,4,5,6,8,13,14]]
#print(device_numbers_dec)

In [23]:
device_max_dec=[]
device_mean_dec = []
device_median_dec = []
for device in device_numbers_dec:
    points_max_dec=result_ping_max_dec.get_points(tags={'SK_PI':str(device)})
    for point in points_max_dec:
        device_median_dec.append(point['median'])
        device_max_dec.append(point['max'])
        device_mean_dec.append(point['mean'])

Plotting mean and max ping latency results for every reporting result in December.

In [24]:
trace1 = go.Bar(
            x=device_numbers_dec,
            y=device_mean_dec,
            name='Mean',
    )
trace2 = go.Bar(
            x=device_numbers_dec,
            y=device_max_dec,
            name='Max',
    
    )
trace3 = go.Bar(
            x=device_numbers_dec,
            y=device_median_dec,
            name='Median',
    
    )
data = [trace1, trace2, trace3]
layout = go.Layout(
       # barmode='stack',
        title="Maximum, mean and median ping delay in miliseconds for the last 4 weeks"
    )

fig = go.Figure(data=data, layout=layout)
iplot(fig)

Device 16 has the highest mean, it needs to be, (all these spikes are also visible in Grafana). Something is going on there. Device 13 looks like it was giving high latency but stopped reporting Dec 4th

### Ping latency grouped by duration
Lets divide ping latencies into 3 groups:
 - group1:  1-40Ms delay
 - group2:  41-100Ms delay
 - group3:  101+Ms delay  
 And calculate percentages of ping latencies in each group for every device:

In [25]:
query_ping_group1= "SELECT COUNT(PING) FROM PING WHERE PING > 0 AND PING < 41 AND time >= now()-4w GROUP BY SK_PI;"
result_ping_group1 = client.query(query_ping_group1)

In [26]:
query_ping_group2= "SELECT COUNT(PING) FROM PING WHERE PING > 40 AND PING < 101 AND  time >= now()-4w  GROUP BY SK_PI;"
result_ping_group2 = client.query(query_ping_group2)

In [27]:
query_ping_group3= "SELECT COUNT(PING) FROM PING WHERE PING > 100  AND  time >= now()-4w  GROUP BY SK_PI;"
result_ping_group3 = client.query(query_ping_group3)

In [28]:
ping_group1=[]
ping_group2 = []
ping_group3 = []
for device in device_numbers_dec:
    points_ping_group1=result_ping_group1.get_points(tags={'SK_PI':str(device)})
    points_ping_group2=result_ping_group2.get_points(tags={'SK_PI':str(device)})
    points_ping_group3=result_ping_group3.get_points(tags={'SK_PI':str(device)})
    point_new=0
    for point in points_ping_group1:
        point_new=point['count']
    ping_group1.append(point_new)
    point_new=0
    for point in points_ping_group2:
        point_new=point['count']
    ping_group2.append(point_new)
    point_new=0
    for point in points_ping_group3:
        point_new=point['count']
    ping_group3.append(point_new)
    

In [29]:
#ping_counts_dec1=list(filter(lambda a: a != 0, ping_counts_dec))
#pingdroprate_counts_dec1=list(filter(lambda a: a != 0, pingdroprate_counts_dec))
#packets_sent_dec = [(a - b) for a, b in zip(ping_counts_dec1, pingdroprate_counts_dec1)]
packets_sent_dec= [(a + b +c) for a, b,c in zip(ping_group1, ping_group2,ping_group3)]
print(ping_group1)
print(ping_group2)
print(ping_group3)
#print(ping_counts_dec1)
#print(pingdroprate_counts_dec1)
print(packets_sent_dec)


[449681, 260633, 414968, 0, 442476, 340265, 449802, 374993, 0, 0]
[312, 35824, 29787, 291208, 3717, 127227, 144, 20759, 328399, 245455]
[0, 3619, 3640, 483, 1, 486, 10, 45265, 1916, 2039]
[449993, 300076, 448395, 291691, 446194, 467978, 449956, 441017, 330315, 247494]


In [42]:
trace1 = go.Bar(
            x=device_numbers_dec,
            y=[(a / b)*100 for a, b in zip(ping_group1, packets_sent_dec)],
            name='percentage of pings with 1-40Ms delay',
    )
trace2 = go.Bar(
            x=device_numbers_dec,
            y=[(a / b)*100 for a, b in zip(ping_group2, packets_sent_dec)],
            name='percentage of pings with 41-100Ms delay',
    
    )
trace3 = go.Bar(
            x=device_numbers_dec,
            y=[(a / b)*100 for a, b in zip(ping_group3, packets_sent_dec)],
            name='percentage of pings with 101+Ms delay',
    
    )
data1 = [trace1, trace2, trace3]

trace4 = go.Bar(
            x=device_numbers_dec,
            y=ping_group1,
            name='number of pings with 1-40Ms delay',
    )
trace5 = go.Bar(
            x=device_numbers_dec,
            y=ping_group2,
            name='number of pings with 41-100Ms delay',
    
    )
trace6 = go.Bar(
            x=device_numbers_dec,
            y=ping_group3,
            name='number of pings with 101+Ms delay',
    
    )
data2 = [trace4, trace5, trace6]

layout1 = go.Layout(
        barmode='stack',
        title=" Ping latency grouped by duration(percentage)"
    )
layout2 = go.Layout(
        barmode='stack',
        title=" Ping latency grouped by duration(actual number)"
    )
fig1 = go.Figure(data=data1, layout=layout1)
fig2 = go.Figure(data=data2, layout=layout2)
iplot(fig1)
iplot(fig2)

### Ping latency grouped by time of the day
We will find mean latency for different time of the day for all the devices in the last 4 weeks:
 - night: 23:00 - 07:00
 - day: 07:00 - 17:00
 - evening: 17:00 - 23:00

Could not figure out how to do this, may be something from here: https://docs.influxdata.com/influxdb/v1.7/query_language/data_exploration/#advanced-group-by-time-syntax

## Ping droprate 
We will calculate the etite number of packets dropped by device using formula:SUM(PING_DROPRATE)/100

In [40]:
#query_dropped_counts = "SELECT SUM(PING_DROPRATE)/100 FROM PING WHERE time >= now()-4w GROUP BY SK_PI ;"
#result_dropped_counts = client.query(query_dropped_counts)

In [41]:
#device_dropped_counts=[]
#for device in device_numbers:
#    points_dropped_counts=result_dropped_counts.get_points(tags={'SK_PI':str(device)})
#    for point in points_dropped_counts:
#        device_dropped_counts.append(point['sum'])
#print(device_dropped_counts)
#print(sum(device_dropped_counts))

[10, 974, 3601, 118815, 1192, 2320, 43, 10628, 1523, 1231]
140337


In [53]:
ping_counts_dec1=list(filter(lambda a: a != 0, ping_counts_dec))
pingdroprate_counts_dec1=list(filter(lambda a: a != 0, pingdroprate_counts_dec))
trace1 = go.Bar(
            x=device_numbers_dec,
            y=[(a / b)*100 for a, b in zip(pingdroprate_counts_dec1, ping_counts_dec1)],
            name='percentage of pings dropped',
    )
trace2 = go.Bar(
            x=device_numbers_dec,
            y=[((b-a) / b)*100 for a, b in zip(pingdroprate_counts_dec1, ping_counts_dec1)],
            name='percentage of pings delievered',
    
    )

data1 = [trace1, trace2]

trace3 = go.Bar(
            x=device_numbers_dec,
            y=pingdroprate_counts_dec1,
            name='number of pings dropped',
    )
trace4 = go.Bar(
            x=device_numbers_dec,
            y=[(b-a)  for a, b in zip(pingdroprate_counts_dec1, ping_counts_dec1)],
            name='number of pings delievered',
    
    )

data2 = [trace3, trace4]

layout1 = go.Layout(
        barmode='stack',
        title="Ping droprate(percentage)"
    )
layout2 = go.Layout(
        #barmode='stack',
        title="Ping droprate(actual number)"
    )
fig1 = go.Figure(data=data1, layout=layout1)
fig2 = go.Figure(data=data2, layout=layout2)
iplot(fig1)
iplot(fig2)

Device #10 and device #16  have the largest number of packets dropped.

In [49]:
#trace = go.Pie(labels=device_numbers, values=device_dropped_counts)
#data=[trace]
#layout = go.Layout(
#        title="Total number of packets dropped by device"
#    )
#fig = go.Figure(data=data, layout=layout)
#iplot(fig)

In [50]:
#query_dropped_counts_dec = "SELECT SUM(PING_DROPRATE)/100 FROM PING WHERE time >= '2018-12-01 00:00:00' GROUP BY SK_PI ;"
#result_dropped_counts_dec = client.query(query_dropped_counts_dec)

In [51]:
#device_dropped_counts_dec=[]
#for device in device_numbers:
#    points_dropped_counts_dec=result_dropped_counts_dec.get_points(tags={'SK_PI':str(device)})
#    point_sum=0
#    for point in points_dropped_counts_dec:
#        point_sum=point['sum']
#    device_dropped_counts_dec.append(point_sum)    

In [52]:
#trace = go.Pie(labels=device_numbers, values=device_dropped_counts_dec)
#data=[trace]
#layout = go.Layout(
#        title="Total number of packets dropped by device in December"
#    )
#fig = go.Figure(data=data, layout=layout)
#iplot(fig)